# Tutorial for CLCLSA using BRCA as an example

### 1. Complete multi-omics classification

In [1]:
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import numpy as np
import pprint

from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from utils.data_utils import one_hot_tensor, prepare_trte_data, get_mask
from networks.models.clcl import CLUECL3
from datetime import datetime
from tqdm import tqdm

/opt/conda/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 1.1 Load data

In [2]:
data_folder = "BRCA"
# training data, testing data, training and testing indices, labels
data_tr_list, data_test_list, trte_idx, labels_trte = prepare_trte_data(data_folder, True)
print(f"total number of subjects = {len(labels_trte)}, training = {data_tr_list[0].shape[0]}, testing = {data_test_list[0].shape[0]}")
print(f"number of omics type = {len(data_tr_list)}")
print(f"shape of each omics in training data:")
for i in range(len(data_tr_list)):
    print(f"shape of {i}-th omics training data = {data_tr_list[i].shape}")
print(f"shape of each omics in testing data:")
for i in range(len(data_tr_list)):
    print(f"shape of {i}-th omics testing data= {data_test_list[i].shape}")
print(f"data type = {type(data_tr_list[0])}")

total number of subjects = 875, training = 612, testing = 263
number of omics type = 3
shape of each omics in training data:
shape of 0-th omics training data = torch.Size([612, 1000])
shape of 1-th omics training data = torch.Size([612, 503])
shape of 2-th omics training data = torch.Size([612, 1000])
shape of each omics in testing data:
shape of 0-th omics testing data= torch.Size([263, 1000])
shape of 1-th omics testing data= torch.Size([263, 503])
shape of 2-th omics testing data= torch.Size([263, 1000])
data type = <class 'torch.Tensor'>


In [3]:
# pre-processing data
num_class = len(np.unique(labels_trte))       # number of class in the multi-omics dataset
labels_tr_tensor = torch.LongTensor(labels_trte[trte_idx["tr"]])
onehot_labels_tr_tensor = one_hot_tensor(labels_tr_tensor, num_class)
labels_tr_tensor = labels_tr_tensor.cuda()
onehot_labels_tr_tensor = onehot_labels_tr_tensor.cuda()

### 1.2 Build moodel

In [4]:
dim_list = [x.shape[1] for x in data_tr_list] # dimension of each omics data
hidden_dim = "200"                            # number of units in the hidden_layer
hidden_dim = [int(x) for x in hidden_dim.split(",")]
dropout = 0.5                                 # dropout rate
cross_omics_prediction_layers = "64,32"       # hyper parameters for cross-omics latent feature impuatation
prediction = {i: [int(x) for x in cross_omics_prediction_layers.split(",")] for i in range(3)}
model = CLUECL3(dim_list, hidden_dim, num_class, dropout, prediction)

### 1.3 Training and evaluation

In [5]:
# define optimizer and move model to GPU
learning_rate = 1e-4                # learning rate for optimization
step_size = 500                     # step size for the scheduler
num_epoch = 2500                    # number of total training epochs
test_inverval = 500                 # number of epoch for testing
missing_rate = 0.                   # set 0 for complete multi-omics setting
device = "cuda"                     # use cuda for GPU training
lambda_al = 0.1                     # balancing factor for auxilary classifier
lambda_co = 0.1                     # balancing factor for cross-omics encoder
lambda_cil = 0.1                    # balancing factor for comtrastive learning

model.cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=0.2)

In [6]:
# function for training one epoch
def train_epoch(model, optimizer, missing_rate, data_tr_list, labels_tr_tensor, print, device, lambda_al, lambda_co, lambda_cil, mask_train=None):
    model.train()
    optimizer.zero_grad()
    if missing_rate > 0:
        loss, _, loss_dict = model.train_missing_cg(data_tr_list, mask_train, labels_tr_tensor, device,
                                                    aux_loss=lambda_al>0, lambda_al=lambda_al,
                                                    cross_omics_loss=lambda_co>0, lambda_col=lambda_co,
                                                    constrastive_instance_loss=lambda_cil>0, lambda_cil=lambda_cil,
                                                    constrastive_cluster_loss=False, lambda_ccl=0.)
    else:
        loss, _, loss_dict = model(data_tr_list, labels_tr_tensor, aux_loss=lambda_al>0, lambda_al=lambda_al)
    
    if print:
        pprint.pprint(loss_dict)
    loss = torch.mean(loss)
    loss.backward()
    optimizer.step()

# function for testing one epoch
def test_epoch(model, missing_rate, data_test_list, device, mask_test=None):
    model.eval()
    with torch.no_grad():
        if missing_rate > 0:
            logit = model.infer_on_missing(data_test_list, mask_test, device)
        else:
            logit = model.infer(data_test_list)
        prob = F.softmax(logit, dim=1).data.cpu().numpy()
    return prob

In [7]:
global_acc = 0.
best_eval = []
best_epoch = 0
print("\nTraining...")
for epoch in tqdm(range(num_epoch + 1)):
    # print_loss = True if epoch % test_inverval == 0 else False
    print_loss = False
    # def train_epoch(model, optimizer, missing_rate, data_tr_list, labels_tr_tensor, print, device, lambda_al, lambda_co, lambda_cil, mask_train=None):
    train_epoch(model, optimizer, missing_rate, data_tr_list, labels_tr_tensor, print_loss, device, lambda_al, lambda_co, lambda_cil)

    scheduler.step()
    if epoch % test_inverval == 0:
        # def test_epoch(model, missing_rate, data_test_list, device, mask_test=None):
        te_prob = test_epoch(model, missing_rate, data_test_list, device)
        if not np.any(np.isnan(te_prob)):
            # print("\nTest: Epoch {:d}".format(epoch))
            if num_class == 2:
                acc = accuracy_score(labels_trte[trte_idx["te"]], te_prob.argmax(1))
                f1 = f1_score(labels_trte[trte_idx["te"]], te_prob.argmax(1))
                auc = roc_auc_score(labels_trte[trte_idx["te"]], te_prob[:, 1])
                # print(f"Test ACC: {acc:.5f}, F1: {f1:.5f}, AUC: {auc:.5f}")
                if acc > global_acc:
                    global_acc, best_epoch = acc, epoch
                    best_eval = [acc, f1, auc]
            else:
                acc = accuracy_score(labels_trte[trte_idx["te"]], te_prob.argmax(1))
                f1w = f1_score(labels_trte[trte_idx["te"]], te_prob.argmax(1), average='weighted')
                f1m = f1_score(labels_trte[trte_idx["te"]], te_prob.argmax(1), average='macro')
                # print(f"Test ACC: {acc:.5f}, F1 weighted : {f1w:.5f}, F1 macro: {f1m:.5f}")
                if acc > global_acc:
                    global_acc = acc
                    best_eval, best_epoch = [acc, f1w, f1m], epoch
print(f"global_acc = {global_acc}, best_acc = {best_eval[0]}, best_weighted_f1 = {best_eval[1]}, best_macro_f1 = {best_eval[2]}")


Training...


100%|██████████| 2501/2501 [00:43<00:00, 56.94it/s]

global_acc = 0.870722433460076, best_acc = 0.870722433460076, best_weighted_f1 = 0.8727098387935339, best_macro_f1 = 0.8358228702640293


### 2. Cmomplete multi-omics classification

### 2.1 Load data

In [17]:
data_folder = "BRCA"
# training data, testing data, training and testing indices, labels
data_tr_list, data_test_list, trte_idx, labels_trte = prepare_trte_data(data_folder, True)
print(f"total number of subjects = {len(labels_trte)}, training = {data_tr_list[0].shape[0]}, testing = {data_test_list[0].shape[0]}")
print(f"number of omics type = {len(data_tr_list)}")
print(f"shape of each omics in training data:")
for i in range(len(data_tr_list)):
    print(f"shape of {i}-th omics training data = {data_tr_list[i].shape}")
print(f"shape of each omics in testing data:")
for i in range(len(data_tr_list)):
    print(f"shape of {i}-th omics testing data= {data_test_list[i].shape}")
print(f"data type = {type(data_tr_list[0])}")

total number of subjects = 875, training = 612, testing = 263
number of omics type = 3
shape of each omics in training data:
shape of 0-th omics training data = torch.Size([612, 1000])
shape of 1-th omics training data = torch.Size([612, 503])
shape of 2-th omics training data = torch.Size([612, 1000])
shape of each omics in testing data:
shape of 0-th omics testing data= torch.Size([263, 1000])
shape of 1-th omics testing data= torch.Size([263, 503])
shape of 2-th omics testing data= torch.Size([263, 1000])
data type = <class 'torch.Tensor'>


In [18]:
# pre-processing data
num_class = len(np.unique(labels_trte))       # number of class in the multi-omics dataset
labels_tr_tensor = torch.LongTensor(labels_trte[trte_idx["tr"]])
onehot_labels_tr_tensor = one_hot_tensor(labels_tr_tensor, num_class)
labels_tr_tensor = labels_tr_tensor.cuda()
onehot_labels_tr_tensor = onehot_labels_tr_tensor.cuda()

### 2.2 Generate in-complete multi-omics data

In [19]:
def get_mask_wrapper(n_views, data_len, missing_rate):
    success = False
    while not success:
        try:
            mask = get_mask(n_views, data_len, missing_rate)
            success = True
        except:
            success = False
    
    return mask

In [20]:
missing_rate = 0.1 # set missing rate
mask = get_mask(3, data_tr_list[0].shape[0], missing_rate)
mask = torch.from_numpy(np.asarray(mask, dtype=np.float32)).to(device)
x1_train = data_tr_list[0] * torch.unsqueeze(mask[:, 0], 1)
x2_train = data_tr_list[1] * torch.unsqueeze(mask[:, 1], 1)
x3_train = data_tr_list[2] * torch.unsqueeze(mask[:, 2], 1)
mask_train = mask
data_tr_list = [x1_train, x2_train, x3_train]
mask = get_mask_wrapper(3, data_test_list[0].shape[0], missing_rate)
mask = torch.from_numpy(np.asarray(mask, dtype=np.float32)).to(device)
x1_test = data_test_list[0] * torch.unsqueeze(mask[:, 0], 1)
x2_test = data_test_list[1] * torch.unsqueeze(mask[:, 1], 1)
x3_test = data_test_list[2] * torch.unsqueeze(mask[:, 2], 1)
mask_test = mask
data_test_list = [x1_test, x2_test, x3_test]

### 2.3 Build moodel

In [23]:
dim_list = [x.shape[1] for x in data_tr_list] # dimension of each omics data
hidden_dim = "200"                            # number of units in the hidden_layer
hidden_dim = [int(x) for x in hidden_dim.split(",")]
dropout = 0.5                                 # dropout rate
cross_omics_prediction_layers = "64,32"       # hyper parameters for cross-omics latent feature impuatation
prediction = {i: [int(x) for x in cross_omics_prediction_layers.split(",")] for i in range(3)}
model = CLUECL3(dim_list, hidden_dim, num_class, dropout, prediction)

model.cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=0.2)

### 2.4 Traning and evaluation

In [24]:
learning_rate = 1e-4                # learning rate for optimization
step_size = 500                     # step size for the scheduler
num_epoch = 2500                    # number of total training epochs
test_inverval = 500                 # number of epoch for testing
missing_rate = 0.                   # set 0 for complete multi-omics setting
device = "cuda"                     # use cuda for GPU training
lambda_al = 0.1                     # balancing factor for auxilary classifier
lambda_co = 0.1                     # balancing factor for cross-omics encoder
lambda_cil = 0.1                    # balancing factor for comtrastive learning

In [25]:
global_acc = 0.
best_eval = []
best_epoch = 0
print("\nTraining...")
for epoch in tqdm(range(num_epoch + 1)):
    # print_loss = True if epoch % test_inverval == 0 else False
    print_loss = False
    # def train_epoch(model, optimizer, missing_rate, data_tr_list, labels_tr_tensor, print, device, lambda_al, lambda_co, lambda_cil, mask_train=None):
    train_epoch(model, optimizer, missing_rate, data_tr_list, labels_tr_tensor, print_loss, device, lambda_al, lambda_co, lambda_cil, mask_train)

    scheduler.step()
    if epoch % test_inverval == 0:
        # def test_epoch(model, missing_rate, data_test_list, device, mask_test=None):
        te_prob = test_epoch(model, missing_rate, data_test_list, device, mask_test)
        if not np.any(np.isnan(te_prob)):
            # print("\nTest: Epoch {:d}".format(epoch))
            if num_class == 2:
                acc = accuracy_score(labels_trte[trte_idx["te"]], te_prob.argmax(1))
                f1 = f1_score(labels_trte[trte_idx["te"]], te_prob.argmax(1))
                auc = roc_auc_score(labels_trte[trte_idx["te"]], te_prob[:, 1])
                # print(f"Test ACC: {acc:.5f}, F1: {f1:.5f}, AUC: {auc:.5f}")
                if acc > global_acc:
                    global_acc, best_epoch = acc, epoch
                    best_eval = [acc, f1, auc]
            else:
                acc = accuracy_score(labels_trte[trte_idx["te"]], te_prob.argmax(1))
                f1w = f1_score(labels_trte[trte_idx["te"]], te_prob.argmax(1), average='weighted')
                f1m = f1_score(labels_trte[trte_idx["te"]], te_prob.argmax(1), average='macro')
                # print(f"Test ACC: {acc:.5f}, F1 weighted : {f1w:.5f}, F1 macro: {f1m:.5f}")
                if acc > global_acc:
                    global_acc = acc
                    best_eval, best_epoch = [acc, f1w, f1m], epoch
print(f"global_acc = {global_acc}, best_acc = {best_eval[0]}, best_weighted_f1 = {best_eval[1]}, best_macro_f1 = {best_eval[2]}")


Training...


100%|██████████| 2501/2501 [00:42<00:00, 59.54it/s]

global_acc = 0.8631178707224335, best_acc = 0.8631178707224335, best_weighted_f1 = 0.8672270264074836, best_macro_f1 = 0.822689924546818
